In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import time

from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import LSTM, Input, Embedding, Lambda, Dense, concatenate, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
import keras.backend as K
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import StratifiedKFold, KFold,cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import mean_squared_error
from math import sqrt
from statistics import mean
from xgboost import XGBRegressor
import lightgbm as lgb

import datetime
from time import time
import gc

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
import lightgbm as lgb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from google.colab import drive

drive.mount('/content/drive')
import os
print(os.listdir("./drive/My Drive/DL3/input"))

# Any results you write to the current directory are saved as output.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['train.csv', 'test.csv', 'solution.csv', 'attributes.csv']


# Read the data

In [13]:
train = pd.read_csv('./drive/My Drive/DL3/input/train.csv', encoding="ISO-8859-1")
test = pd.read_csv('./drive/My Drive/DL3/input/test.csv', encoding="ISO-8859-1")
y_test = pd.read_csv('./drive/My Drive/DL3/input/solution.csv', encoding="ISO-8859-1")
prod_att = pd.read_csv('./drive/My Drive/DL3/input/attributes.csv', encoding="ISO-8859-1")

print('train size', train.shape)
print('test size', test.shape)
print('y_test size', y_test.shape)
print('att size', prod_att.shape)

train size (74067, 5)
test size (166693, 4)
y_test size (166693, 3)
att size (2044803, 3)


# Add test relevence to calculate test MAE

In [0]:
# THIS CELL ADDED ONLY IN THE END AFTER WE GET FINAL MODEL (ON TRAIN SET) 

test = pd.merge(test, y_test, on='id', how='left')
test = test.loc[test['Usage'] != 'Ignored']
test.drop(['Usage'], axis=1, inplace=True)

In [0]:
train_size = train.shape[0]
df_all = pd.concat((train, test), axis=0, ignore_index=True)

In [0]:
df_all.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


In [0]:
prod_att_metrial = prod_att.loc[prod_att['name'] == 'Material'][['product_uid', 'value']].drop_duplicates(['product_uid'])
prod_att_brand = prod_att.loc[prod_att['name'] == 'MFG Brand Name'][['product_uid', 'value']].drop_duplicates(['product_uid'])

def merge_att(att_df):
    global df_all
    df_all = pd.merge(df_all, att_df, on='product_uid', how='left')
    df_all = df_all.fillna('')
    df_all['product_title'] = df_all['product_title'] + ' ' + df_all['value']
    df_all.drop(['value'], axis=1, inplace=True)
    return df_all

df_all = merge_att(prod_att_metrial)
df_all = merge_att(prod_att_brand)

In [0]:
df_all.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle Galvanized S...,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle Galvanized S...,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


# Split product_titile and search_term to chars

In [0]:
char_to_remove = [' ', '{', '}', '"', '(', ')', '.', ',', '&','[',']','`','_','\'', '~', '\\', '-', ':', ';', '=', '?',
                  '\x80',  '\x81', '\x82', '\x84', '\x89', '\x8b', '\x90', '\x93', '\x95', '\x99', '\x9a', '!',
                  '\x9d', '\xa0', '¡', '¢', '®', 'ª', 'À', 'Ã', 'Â', 'È', 'Ê', 'Ë', 'Ï', 'Ò', 'Û', 
                  'Ü', 'â', 'ã', 'å', 'è', '÷', '$', '*']

def lower_char(char):
    if char >= 'A' and char <= 'Z':
        return char.lower()
    else:
        return char

df_all['product_title'] = df_all['product_title'].apply(lambda sent: [lower_char(character) for character in sent if character not in char_to_remove])
df_all['search_term'] = df_all['search_term'].apply(lambda sent: [lower_char(character) for character in sent if character not in char_to_remove])

In [0]:
df_all.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,"[s, i, m, p, s, o, n, s, t, r, o, n, g, t, i, ...","[a, n, g, l, e, b, r, a, c, k, e, t]",3.00
1,3,100001,"[s, i, m, p, s, o, n, s, t, r, o, n, g, t, i, ...","[l, b, r, a, c, k, e, t]",2.50
2,9,100002,"[b, e, h, r, p, r, e, m, i, u, m, t, e, x, t, ...","[d, e, c, k, o, v, e, r]",3.00
3,16,100005,"[d, e, l, t, a, v, e, r, o, 1, h, a, n, d, l, ...","[r, a, i, n, s, h, o, w, e, r, h, e, a, d]",2.33
4,17,100005,"[d, e, l, t, a, v, e, r, o, 1, h, a, n, d, l, ...","[s, h, o, w, e, r, o, n, l, y, f, a, u, c, e, t]",2.67


# Split back to train and test sets

In [18]:
df_train = df_all.iloc[:train_size]
df_test = df_all.iloc[train_size:]
df_test.reset_index(inplace=True, drop=True)

y_data = df_train['relevance']

# split to train and validation sets
train_indx = int(0.8*df_train.shape[0])
train = df_train[:train_indx]
val = df_train[train_indx:]
print('train size', train.shape)
print('val size', val.shape)
print('test size', df_test.shape)

train size (59253, 5)
val size (14814, 5)
test size (112067, 5)


# Baseline Model- Common Characters

In [0]:
#calculate char match between search term and product title
def commonCharsUnion(search,title):
    searchSet = set(search)
    searchLen = len(searchSet)
    if searchLen==0:
      return 0
    titleSet = set(title)
    commonLen = len(titleSet.intersection(searchSet))
    unionLen=len(titleSet.union(searchSet))
    return 3*commonLen/unionLen

#calculate char match for data
def calculateCommonChars(data):
    l_data =len(data)
    res_array= np.zeros(l_data)
    for i in range(0, l_data):
        res_array[i] = commonCharsUnion(data.iloc[i]['search_term'],data.iloc[i]['product_title'])
    return res_array
   
    

arr=calculateCommonChars(train)
print(arr)
train_mae = mean_absolute_error(train['relevance'], arr)
print('train MAE:',train_mae)
train_rmse =sqrt( mean_squared_error(train['relevance'], arr))
print('train RMSE:',train_rmse)



arrtest=calculateCommonChars(test)
print(arr)
test_mae =  mean_absolute_error(test['relevance'], arrtest)
print('test MAE:',test_mae)
test_rmse =sqrt(mean_squared_error(test['relevance'], arrtest))
print('test RMSE:',test_rmse)



arrval=calculateCommonChars(val)
print(arr)
val_mae =  mean_absolute_error(val['relevance'], arrval)
print('val MAE:',val_mae)
val_rmse =sqrt(mean_squared_error(val['relevance'], arrval))
print('val RMSE:',val_rmse)

[1.         0.71428571 0.875      ... 1.2        2.05263158 1.42857143]
train MAE: 1.0445731807413108
train RMSE: 1.1873387629428342
[1.         0.71428571 0.875      ... 1.2        2.05263158 1.42857143]
test MAE: 1.351267225757503
test RMSE: 1.4685515611296533
[1.         0.71428571 0.875      ... 1.2        2.05263158 1.42857143]
val MAE: 0.760767448261394
val RMSE: 0.8979442501780921


In [0]:
#calculate char match between search term and product title
def commonChars(search,title):
    searchSet = set(search)
    searchLen = len(searchSet)
    if searchLen==0:
      return 0
    titleSet = set(title)
    commonLen = len(titleSet.intersection(searchSet))
    return 3*commonLen/searchLen
  
#calculate char match for data
def calculateCommonCharsSearch(data):
    l_data =len(data)
    res_array= np.zeros(l_data)
    for i in range(0, l_data):
        res_array[i] = commonChars(data.iloc[i]['search_term'],data.iloc[i]['product_title'])
    return res_array


arr=calculateCommonCharsSearch(train)
print(arr)
train_mae = mean_absolute_error(train['relevance'], arr)
print('train MAE:',train_mae)
train_rmse =sqrt( mean_squared_error(train['relevance'], arr))
print('train RMSE:',train_rmse)



arrtest=calculateCommonCharsSearch(test)
print(arr)
test_mae =  mean_absolute_error(test['relevance'], arrtest)
print('test MAE:',test_mae)
test_rmse =sqrt(mean_squared_error(test['relevance'], arrtest))
print('test RMSE:',test_rmse)



arrval=calculateCommonCharsSearch(val)
print(arr)
val_mae =  mean_absolute_error(val['relevance'], arrval)
print('val MAE:',val_mae)
val_rmse =sqrt(mean_squared_error(val['relevance'], arrval))
print('val RMSE:',val_rmse)


[2.1        1.875      3.         ... 3.         2.78571429 3.        ]
train MAE: 0.48131289650384684
train RMSE: 0.6674856801889621
[2.1        1.875      3.         ... 3.         2.78571429 3.        ]
test MAE: 0.480135382550569
test RMSE: 0.6317887456400187
[2.1        1.875      3.         ... 3.         2.78571429 3.        ]
val MAE: 0.6226011633875516
val RMSE: 0.7875652125747274


#Baseline Model- Common N-Grams

In [0]:
ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2,2),binary=True)
#calculate common nGrams
def commonNGrams(data):
    l_data =len(data)
    res_array= np.zeros(l_data)
    for i in range(0, l_data):
        counts = ngram_vectorizer.fit_transform([''.join(data.iloc[i]['product_title']),''.join(data.iloc[i]['search_term'])])
        arr = counts.toarray().astype(int)
        countCommonNGrams=0
        for j in range(0, len(arr[0])):
            if(arr[0,j]+arr[1,j]==2):
                countCommonNGrams = countCommonNGrams + 1
        numOfNGramsSearch=sum(arr[1])
        res_array[i] = (countCommonNGrams/numOfNGramsSearch)*3
    return res_array
  
  
arr=commonNGrams(train)
print(arr)
train_mae = mean_absolute_error(train['relevance'], arr)
print('train MAE:',train_mae)
train_rmse =sqrt(mean_squared_error(train['relevance'], arr))
print('train RMSE:',train_rmse)



arrtest=commonNGrams(test)
print(arr)
test_mae = mean_absolute_error(test['relevance'], arrtest)
print('test MAE:',test_mae)
test_rmse =sqrt(mean_squared_error(test['relevance'], arrtest))
print('test RMSE:',test_rmse)



arrval=commonNGrams(val)
print(arr)
val_mae =mean_absolute_error(val['relevance'], arrval)
print('val MAE:',val_mae)
val_rmse =sqrt(mean_squared_error(val['relevance'], arrval))
print('val RMSE:',val_rmse)

[0.92307692 0.         2.66666667 ... 2.33333333 1.95       1.4       ]
train MAE: 0.7040352353901828
train RMSE: 0.8816330903787409
[0.92307692 0.         2.66666667 ... 2.33333333 1.95       1.4       ]
test MAE: 0.5414038178983509
test RMSE: 0.722429612975288
[0.92307692 0.         2.66666667 ... 2.33333333 1.95       1.4       ]
val MAE: 0.5841915490281006
val RMSE: 0.7283313110216336


#Baseline Model- Mean Relevance Train

In [23]:
avg_rate_train=mean(train['relevance'])

def getMeanRate(avg_rate_train,data):
    l_data =len(data)
    res_array=np.empty(l_data)
    res_array.fill(avg_rate_train)
    return res_array
  
  
arr=getMeanRate(avg_rate_train,train)
print(arr)
train_mae = mean_absolute_error(train['relevance'], arr)
print('train MAE:',train_mae)
train_rmse =sqrt(mean_squared_error(train['relevance'], arr))
print('train RMSE:',train_rmse)



arrtest=getMeanRate(avg_rate_train,test)
print(arr)
test_mae = mean_absolute_error(test['relevance'], arrtest)
print('test MAE:',test_mae)
test_rmse =sqrt(mean_squared_error(test['relevance'], arrtest))
print('test RMSE:',test_rmse)



arrval=getMeanRate(avg_rate_train,val)
print(arr)
val_mae =mean_absolute_error(val['relevance'], arrval)
print('val MAE:',val_mae)
val_rmse =sqrt(mean_squared_error(val['relevance'], arrval))
print('val RMSE:',val_rmse)

[2.41906368 2.41906368 2.41906368 ... 2.41906368 2.41906368 2.41906368]
train MAE: 0.44078251449705097
train RMSE: 0.5308384846455229
[2.41906368 2.41906368 2.41906368 ... 2.41906368 2.41906368 2.41906368]
test MAE: 0.4411998487567881
test RMSE: 0.5366571635786496
[2.41906368 2.41906368 2.41906368 ... 2.41906368 2.41906368 2.41906368]
val MAE: 0.4398050173264314
val RMSE: 0.5527396043931081
